# Generic playspace

# Parsing `file_id`

In [ ]:
files = [
    "bz687a.pm107912_mon.nc",
    "bz687a.p7107912_mon.nc",
    "iceh.1917-05.nc",
    "iceh.1917-05-daily.nc",
    "ocean_month.nc",
    "ocean.nc",
    "ocean-3d-v-1-monthly-pow02-ym_1958_04.nc",
    "ocean-2d-sfc_salt_flux_restore-1-monthly-mean-ym_1958_04.nc",
    "PI-GWL-B2035.pe-109904_dai.nc",
    "PI-GWL-B2035.pa-109904_mon.nc",
    "PI-1pct-02.pe-011802_dai.nc_dai.nc",
    "iceh.0101-01.nc",
    "ocean_bgc_ann.nc-01551231",
    "ocean_bgc_daily.nc-02971231",
    "ocean_bgc_mth.nc-03011231",
    "ocean_bgc.nc-02011231",
    "ocean_daily.nc-02531231",
    "ocean_month.nc-01991231",
    "ocean_scalar.nc-01851231",
]

In [ ]:
import re
from pathlib import Path

for file in files:
filename = Path(file).stem
# Remove dates dddd-dd, dddddd, dddddddd, starting from right
file_id = re.sub(r"(\d{4}[-_]\d{2}|\d{6}|\d{8})(([^0-9]|$))(.*)$", r"\3\4", filename)
# Enforce Python characters
file_id = re.sub(r"[-.]", "_", file_id)
# Remove any double or dangling _
file_id = re.sub(r"__", "_", file_id).rstrip("_")
    print(file_id)

In [ ]:
import re
from pathlib import Path

for file in files:
    filename = Path(file).stem

    file_id = re.sub(
        r"[-.]", 
        "_",
        re.sub(
            r"[-_.](\d{4}[-_]\d{2}|\d{6}|\d{8})", r'boo\2', filename
        )
    )
    print(file, file_id)